In [3]:
import requests
import json

# Your API key and scoring URL
API_KEY = "N4UpVDp6brxngLCy04VuAa7v6UBWDoL7JDFHV716664n"
SCORING_URL = "https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/d59c7aef-1420-45ed-9369-0461e4da69f8/predictions?version=2021-05-01"

# Get IAM Token
token_url = "https://iam.cloud.ibm.com/identity/token"
token_data = {
    "apikey": API_KEY,
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey"
}
token_headers = {"Content-Type": "application/x-www-form-urlencoded"}
token_response = requests.post(token_url, data=token_data, headers=token_headers)
ml_token = token_response.json()["access_token"]

# Fields in exact order from your training CSV
fields = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'
]

# Example row: Normal traffic (from first row of Train_data.csv)
normal_values = [
    0, 'tcp', 'ftp_data', 'SF', 491, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    2, 2, 0, 0, 0, 0, 1, 0, 0, 150, 25, 0.17, 0.03, 0.17, 0, 0, 0, 0.05, 0
]

# Example row: Anomalous traffic (third row of Train_data.csv)
anomalous_values = [
    0, 'tcp', 'private', 'S0', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    123, 6, 1, 1, 0, 0, 0.05, 0.07, 0, 255, 26, 0.1, 0.05, 0, 0, 1, 1, 0, 0
]

payload = {
    "input_data": [{
        "fields": fields,
        "values": [normal_values, anomalous_values]  # sending both at once
    }]
}

# Send request
response = requests.post(
    SCORING_URL,
    json=payload,
    headers={
        "Authorization": f"Bearer {ml_token}",
        "Content-Type": "application/json"
    }
)

# Print predictions
print(json.dumps(response.json(), indent=2))


{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          "normal",
          [
            0.0,
            1.0
          ]
        ],
        [
          "anomaly",
          [
            1.0,
            0.0
          ]
        ]
      ]
    }
  ]
}
